# Setup

## Importando bibliotecas

In [2]:
import pandas as pd
import re
import os
import collections
import nltk
import urllib
import bs4 as bs
import re
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy
from rouge_score import rouge_scorer
from numpy import dot
from numpy.linalg import norm

## Leitura da Base

### Leitura de Artigos

In [3]:
list_of_files = os.listdir('../top1000_complete')

In [4]:
#Leitura dos artigos
list_of_files2= []
for i in list_of_files:
    nome = '../top1000_complete/' + i + '/' + 'Documents_xml/' +os.listdir(f'../top1000_complete/{i}/Documents_xml')[0]
    list_of_files2.append(nome)
    
    
lines = []
for file in list_of_files2:
    f= open(file,'r')
    lines.append(f.readlines())
    f.close()

### Leitura dos Sumários

In [5]:
#Leitura do padrão ouro
list_of_files3= []
for i in list_of_files:
    nome = '../top1000_complete/' + i + '/' + 'summary/' +os.listdir(f'../top1000_complete/{i}/summary')[0]
    list_of_files3.append(nome)
    
    
lines3 = []
for file in list_of_files3:
    f= open(file,'r',encoding='utf-8')
    lines3.append(f.readlines())
    f.close()

## Seleção da base
Identificando quais papers possuem as três seções `Resumo`, `Introdução` e `Conclusão`

In [59]:
paper1= lines

In [7]:
summary1= lines3[2]

In [60]:
lista_papers=[]
for j in paper1:
    texto=''
    for i in j:
        texto = texto + str(i)
    lista_papers.append(texto)

texto=''
for i in paper1:
    texto = texto + str(i)

In [98]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*Introduction.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';','', txt)
        txt = re.sub(':','', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = re.sub(r'small\&',"",txt)
        txt = re.sub(r'tagger&amp',"",txt)
        txt = re.sub(r';quot',"",txt)
        txt = re.sub(r'\&amp',"",txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub('\.',txt)
        txt = txt.lower()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = txt.strip()
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        return txt
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<ABSTRACT>)(.+?)(</ABSTRACT>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';',' ;', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = txt.lower()
        txt = re.sub('\.',txt)
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter = re.finditer('(<SECTION.*Conclusion.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';','', txt)
        txt = re.sub(':','', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = re.sub(r'small\&',"",txt)
        txt = re.sub(r'tagger&amp',"",txt)
        txt = re.sub(r';quot',"",txt)
        txt = re.sub(r'\&amp',"",txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub('\.',txt)
        txt = txt.lower()
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = txt.strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        return txt
    return txt

In [63]:
#Aplicando indices aos papers
indices           = range(0,len(lista_papers))
papers_com_indice = dict(zip(indices, lista_papers))

In [64]:
introducao_papers=[]
x=[]
for i,j in zip(list(papers_com_indice.keys()),list(papers_com_indice.values())):
    try:
        introducao_papers.append(preprocessa_intro(j))
        x.append(i)
    except:
        None

In [65]:
 filtro_tem_introducao = dict((key,value) for key, value in papers_com_indice.items() if key  in  x)

In [66]:
abstract_papers=[]
x=[]
for i,j in zip(list(filtro_tem_introducao.keys()),list(filtro_tem_introducao.values())):
    try:
        abstract_papers.append(preprocessa_abstract(j))
        x.append(i)
    except:
        None

In [67]:
filtro_tem_introducao_e_abstract = dict((key,value) for key, value in papers_com_indice.items() if key  in  x)

In [92]:
conclusion_papers=[]
indices_conclusao=[]
for i,j in zip(list(filtro_tem_introducao_e_abstract.keys()),list(filtro_tem_introducao_e_abstract.values())):
    try:
        conclusion_papers.append(preprocessa_conclusion(j))
        indices_conclusao.append(i)
    except:
        None

In [72]:
print('qtd de papers com introdução:',len(papers_com_indice.keys()))
print('qtd de papers com introdução:',len(introducao_papers))
print('qtd de papers com introdução e abstract:',len(abstract_papers))
print('qtd de papers com introduçã, abstract e conclusão:',len(conclusion_papers))

qtd de papers com introdução: 1009
qtd de papers com introdução: 744
qtd de papers com introdução e abstract: 728
qtd de papers com introduçã, abstract e conclusão: 571


In [93]:
total_papers = dict((key,value) for key, value in papers_com_indice.items() if key  in indices_conclusao)

In [88]:
#Definição do abstract
abstract=[]
for i in total_papers.values():
    abstract.append(preprocessa_abstract(i))

In [89]:
#Definição da Introdução
intro=[]
for i in total_papers.values():
    intro.append(preprocessa_intro(i))

In [94]:
#Definição da Conclusao
conclusion=[]
for i in total_papers.values():
    conclusion.append(preprocessa_conclusion(i))

In [13]:
token_abstract   = []
for texto in total_papers.values():
    token_abstract.append(preprocessa_abstract(texto).split())

In [95]:
token_intro   = []
for texto in total_papers.values():
    token_intro.append(preprocessa_intro(texto).split())

In [96]:
token_conclusion   = []
for texto in total_papers.values():
    token_conclusion.append(preprocessa_conclusion(texto).split())

In [97]:
for i in token_abstract[0:2]:
    print(i)

["trigrams'n'tags", 'is', 'an', 'efficient', 'statistical', 'part-of-speech', 'tagger.', 'contrary', 'to', 'claims', 'found', 'elsewhere', 'in', 'the', 'literature', 'we', 'argue', 'that.tagger', 'based', 'on', 'markov', 'models', 'performs', 'at', 'least', 'as', 'well', 'as', 'other', 'current', 'approaches', 'including', 'the', 'maximum', 'entropy', 'framework..recent', 'comparison', 'has', 'even', 'shown', 'that', 'tnt', 'performs', 'significantly', 'better', 'for', 'the', 'tested', 'corpora.', 'we', 'describe', 'the', 'basic', 'model', 'of', 'tnt', 'the', 'techniques', 'used', 'for', 'smoothing', 'and', 'for', 'handling', 'unknown', 'words.', 'furthermore', 'we', 'present', 'evaluations', 'on', 'two', 'corpora.']
['this', 'paper', 'describes.method', 'for', 'linear', 'text', 'segmentation', 'which', 'is', 'twice', 'as', 'accurate', 'and', 'over', 'seven', 'times', 'as', 'fast', 'as', 'the', 'state-of-the-art.inter-sentence', 'similarity', 'is', 'replaced', 'by', 'rank', 'in', 'the'

In [103]:
tokens = token_abstract+token_intro+token_conclusion

In [104]:
model = Word2Vec(tokens, min_count=1)

In [105]:
len(model.wv.key_to_index)

123347

In [109]:
model.wv.key_to_index['are']

9

model.wv.key_to_index

frases_abstract = abstract.split('.')
palavras_frases=[]
for i in frases_abstract:
    palavras_frases.append(i.split(' '))
palavras_frases

## Definição de Funções

In [110]:
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

In [124]:
def funcao_frase_palavra(abstract):
    """input: lista de frases
        output: lista de palavras de cada frase"""
    frases_abstract = abstract.split('.')
    palavras_frases=[]
    for i in frases_abstract:
        palavras_frases.append(i.split(' '))
    try:
        palavras_frases.remove('')
    except:
        return palavras_frases
    return palavras_frases

In [18]:
def vetor_frases(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model.wv[j]
            except:
                soma=soma+0
        x.append(soma)
    return x

In [19]:
def funcao_sim_coseno(a,b):
    valor_coseno = (numpy.dot(a, b))/(numpy.linalg.norm(a)* numpy.linalg.norm(b))
    if valor_coseno is None:
        return 0
    else:
        return valor_coseno

In [23]:
def funcao_sim_coseno(a,b):
    if ((numpy.linalg.norm(a)==0) |(numpy.linalg.norm(b)==0)):
        return 0
    else:
        valor_coseno = (numpy.dot(a, b))/(numpy.linalg.norm(a)* numpy.linalg.norm(b))
        return valor_coseno

In [26]:
def dicionario_similaridade(vetor_a,vetor_b):
    """Input: vetor correspondente às frases das seções a e b
    Output: 
    """
    dicionario={}
    for i in range(len(vetor_a)):
        for j in range(len(vetor_b)):
            dicionario[i,j]= funcao_sim_coseno(vetor_a[i],vetor_b[j])
    return dicionario

# Vetor de frases

In [114]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*Introduction.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';','', txt)
        txt = re.sub(':','', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = re.sub(r'small\&',"",txt)
        txt = re.sub(r'tagger&amp',"",txt)
        txt = re.sub(r';quot',"",txt)
        txt = re.sub(r'\&amp',"",txt)
        txt = re.sub(' +', ' ', txt)
        txt = txt.lower()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = txt.strip()
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        return txt
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<ABSTRACT>)(.+?)(</ABSTRACT>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';',' ;', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = txt.lower()
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter = re.finditer('(<SECTION.*Conclusion.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';','', txt)
        txt = re.sub(':','', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = re.sub(r'small\&',"",txt)
        txt = re.sub(r'tagger&amp',"",txt)
        txt = re.sub(r';quot',"",txt)
        txt = re.sub(r'\&amp',"",txt)
        txt = re.sub(' +', ' ', txt)
        txt = txt.lower()
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = txt.strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        return txt
    return txt

In [154]:
def vetor_frases(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model.wv[j]
            except:
                soma=soma+0
        x.append(soma)
    return x

In [164]:
def dicionario_similaridade(vetor_a,vetor_b):
    """Input: vetor correspondente às frases das seções a e b
    Output: 
    """
    dicionario={}
    for i in range(len(vetor_a)):
        for j in range(len(vetor_b)):
            dicionario[i,j]= funcao_sim_coseno(vetor_a[i],vetor_b[j])
    return dicionario

In [171]:
def funcao_sim_coseno(a,b):
    if ((numpy.linalg.norm(a)==0) |(numpy.linalg.norm(b)==0)):
        return 0
    else:
        valor_coseno = (numpy.dot(a, b))/(numpy.linalg.norm(a)* numpy.linalg.norm(b))
        return valor_coseno

# Aplicando

In [165]:
try2 = funcao_frase_palavra(abstract[0])

In [166]:
palavras_abstract = remove_values_from_list(try2, [''])

In [167]:
vetor_abstract    = vetor_frases(palavras_abstract)

In [168]:
palavras_intro = funcao_frase_palavra(intro[0])
palavras_intro = remove_values_from_list(palavras_intro, [''])
vetor_intro    = vetor_frases(palavras_intro)

In [169]:
palavras_conclusion = funcao_frase_palavra(conclusion[0])
palavras_conclusion = remove_values_from_list(palavras_conclusion, [''])
vetor_conclusion    = vetor_frases(palavras_conclusion)

In [172]:
dicionario = dicionario_similaridade(vetor_abstract, vetor_intro)

In [173]:
dicionario

{(0, 0): 0.63350624,
 (0, 1): 0.7179656,
 (0, 2): 0.699696,
 (0, 3): 0.6022672,
 (0, 4): 0.45069775,
 (0, 5): 0.4719327,
 (0, 6): 0.6292231,
 (0, 7): 0.56552005,
 (0, 8): 0.5293787,
 (0, 9): 0.67780405,
 (0, 10): 0.71689826,
 (0, 11): 0.6157202,
 (0, 12): 0.40322313,
 (0, 13): 0.5034377,
 (0, 14): 0.70514774,
 (0, 15): 0.4914741,
 (0, 16): 0.49190983,
 (0, 17): 0.527207,
 (0, 18): 0.70384824,
 (0, 19): 0.5452292,
 (0, 20): 0.64986145,
 (0, 21): 0.68420947,
 (0, 22): 0.6046043,
 (0, 23): 0.6519313,
 (0, 24): 0.66439277,
 (0, 25): 0.66808933,
 (0, 26): 0.49999288,
 (0, 27): 0.6261904,
 (0, 28): 0.6146812,
 (0, 29): 0.6012775,
 (0, 30): 0.6147374,
 (0, 31): 0.5938282,
 (0, 32): 0.5438574,
 (0, 33): 0.67976594,
 (0, 34): 0.40149057,
 (0, 35): 0.6169873,
 (0, 36): 0.79474854,
 (0, 37): 0.5419512,
 (0, 38): 0.5446912,
 (0, 39): 0.58619744,
 (0, 40): 0.5532664,
 (0, 41): 0.43225262,
 (0, 42): 0.50647944,
 (0, 43): 0.5890183,
 (0, 44): 0.5290954,
 (0, 45): 0.33332938,
 (0, 46): 0.524503,
 (0, 

In [174]:
top_3_indices = sorted(dicionario, key=dicionario.get, reverse=True)[:3]
top_3_indices

[(1, 298), (3, 11), (4, 130)]

In [175]:
for i in top_3_indices:
    print(dicionario[i])

0.929208
0.924776
0.92446077


In [183]:
palavras_abstract[4]

['',
 'we',
 'describe',
 'the',
 'basic',
 'model',
 'of',
 'tnt',
 'the',
 'techniques',
 'used',
 'for',
 'smoothing',
 'and',
 'for',
 'handling',
 'unknown',
 'words']

In [184]:
palavras_intro[130]

['context-independent',
 'approach',
 'for',
 '0&#8222',
 'as',
 'we',
 'did',
 'for',
 'the',
 'contextual',
 'weights',
 'a']

In [185]:
x=[]
for i in range(len(top_3_indices)):
    x.append(top_3_indices[i][0])
indice_abstract = list(set(x))

In [186]:
indice_abstract

[1, 3, 4]

In [187]:
vetor_frases_objetivo_1=[]
for i in range(len(indice_abstract)):
    vetor_frases_objetivo_1.append(vetor_intro[indice_abstract[i]])
vetor_frases_objetivo_1

[array([ 9.76164222e-01, -1.54301739e+00, -1.15254784e+00, -3.48976135e+00,
         5.48465586e+00, -2.86121511e+00, -2.23197043e-03,  6.75335646e+00,
        -1.61694407e+00, -6.83200932e+00,  2.73714375e+00, -1.07148552e+00,
        -2.93583632e+00,  6.55707419e-01,  5.37931347e+00,  4.19831944e+00,
         1.68964207e+00, -1.01991653e+00,  9.03090775e-01, -3.11060810e+00,
         4.84238535e-01,  4.03218806e-01,  1.11081541e+00,  4.82035995e-01,
         1.35956562e+00,  6.09692216e-01, -4.18299055e+00, -2.62711549e+00,
         1.18423367e+00,  1.75045812e+00,  4.01760876e-01, -1.57490826e+00,
        -1.91594958e+00,  5.24013710e+00, -4.23396444e+00, -9.19395626e-01,
         4.21606213e-01, -5.07869673e+00, -5.58111238e+00, -4.56943750e+00,
        -3.16378146e-01, -3.73953700e+00,  1.33787632e+00, -4.47359037e+00,
         1.98470855e+00,  2.27969074e+00, -2.88339996e+00, -2.16939330e+00,
        -6.85466886e-01,  1.74061394e+00,  3.05320692e+00, -3.41664124e+00,
        -1.3

In [188]:
dicionario = dicionario_similaridade(vetor_abstract, vetor_conclusion)

In [189]:
top_3_indices = sorted(dicionario, key=dicionario.get, reverse=True)[:3]
top_3_indices

[(1, 11), (1, 4), (1, 5)]

In [196]:
palavras_conclusion[11]

['comparsion',
 'of',
 'the',
 'results',
 'presented',
 'here',
 'with',
 'those',
 'in',
 'the',
 'maximum',
 'entropy',
 'framework',
 'seems',
 'to',
 'be',
 'the',
 'only',
 'other',
 'approach',
 'yielding',
 'comparable',
 'results',
 'to',
 'the',
 'one',
 'presented',
 'here']

In [197]:
palavras_abstract[1]

['',
 'contrary',
 'to',
 'claims',
 'found',
 'elsewhere',
 'in',
 'the',
 'literature',
 'we',
 'argue',
 'that']

In [192]:
for i in top_3_indices:
    print(dicionario[i])

0.929208
0.9237128
0.9056458


In [193]:
x=[]
for i in range(len(top_3_indices)):
    x.append(top_3_indices[i][0])
indice_abstract = list(set(x))

In [194]:
vetor_frases_objetivo_1=[]
for i in range(len(indice_abstract)):
    vetor_frases_objetivo_1.append(vetor_intro[indice_abstract[i]])
vetor_frases_objetivo_1

[array([ 9.76164222e-01, -1.54301739e+00, -1.15254784e+00, -3.48976135e+00,
         5.48465586e+00, -2.86121511e+00, -2.23197043e-03,  6.75335646e+00,
        -1.61694407e+00, -6.83200932e+00,  2.73714375e+00, -1.07148552e+00,
        -2.93583632e+00,  6.55707419e-01,  5.37931347e+00,  4.19831944e+00,
         1.68964207e+00, -1.01991653e+00,  9.03090775e-01, -3.11060810e+00,
         4.84238535e-01,  4.03218806e-01,  1.11081541e+00,  4.82035995e-01,
         1.35956562e+00,  6.09692216e-01, -4.18299055e+00, -2.62711549e+00,
         1.18423367e+00,  1.75045812e+00,  4.01760876e-01, -1.57490826e+00,
        -1.91594958e+00,  5.24013710e+00, -4.23396444e+00, -9.19395626e-01,
         4.21606213e-01, -5.07869673e+00, -5.58111238e+00, -4.56943750e+00,
        -3.16378146e-01, -3.73953700e+00,  1.33787632e+00, -4.47359037e+00,
         1.98470855e+00,  2.27969074e+00, -2.88339996e+00, -2.16939330e+00,
        -6.85466886e-01,  1.74061394e+00,  3.05320692e+00, -3.41664124e+00,
        -1.3

In [195]:
for i in indice_abstract:
    print(frases_abstract[i])

NameError: name 'frases_abstract' is not defined

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [ ]:
rouge_score_frases = {}
for i in indice_abstract:
    for j in range(len(summary1)):
        #print(scorer.score(frases_abstract[i],summary1[j])['rouge1'].fmeasure)
        rouge_score_frases[i,j] = scorer.score(frases_abstract[i],summary1[j])['rouge1'].fmeasure

In [ ]:
rouge_score_frases

In [ ]:
frases_abstract[1]

In [ ]:
frases_abstract[2]

In [ ]:
summary1

In [48]:
summary1[2]

'Inter-sentence similarity is replaced by rank in the local context.\n'

In [49]:
frases_abstract

['this paper describes',
 'method for linear text segmentation which is twice as accurate and over seven times as fast as the state-of-the-art',
 'inter-sentence similarity is replaced by rank in the local context',
 ' boundary locations are discovered by divisive clustering',
 '']

In [50]:
summary1

['Advances In Domain Independent Linear Text Segmentation\n',
 'This paper describes a method for linear text segmentation which is twice as accurate and over seven times as fast as the state-of-the-art (Reynar, 1998).\n',
 'Inter-sentence similarity is replaced by rank in the local context.\n',
 'Boundary locations are discovered by divisive clustering.\n',
 'We design an artificial dataset, built by concatenating short pieces of texts that have been extracted from the Brown corpus.\n']

In [51]:
abstract

'this paper describes.method for linear text segmentation which is twice as accurate and over seven times as fast as the state-of-the-art.inter-sentence similarity is replaced by rank in the local context. boundary locations are discovered by divisive clustering.'

from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')

scores